In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10
MIN_number_of_attributes = 2
MAX_number_of_attributes = 35
number_of_NN_repetitions = 10
#number_of_attributes = 20

print(tf.__version__)

2.8.0


In [ ]:
path_training ="/content/drive/MyDrive/network-traffic-classification-main/csv-files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/network-traffic-classification-main/csv-files/02_Atributos_Weve.csv"
df_raw = pd.read_csv(path_training)
df_test_raw = pd.read_csv(path_test)
df_raw
target = df_raw.pop('target')
server_port = df_raw.pop('ServerPort')
client_port = df_raw.pop('ClientPort')

target_test = df_test_raw.pop('target')
server_port_test = df_test_raw.pop('ServerPort')
client_port = df_test_raw.pop('ClientPort')

#attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/JSON-ATTRIBUTES-FILE-HERE"
attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/last-run"
with open(attr_title, "r") as f:
  rep = json.load(f)
  print(rep['sorted_attributes_by_weights']) 

attributes_array = rep['sorted_attributes_by_weights']      

['83', '82', '79', '90', '84', '10', '17', '29', '88', '59', '87', '89', '81', '60', '15', '45', '62', '47', '64', '32', '48', '36', '37', '91', '24', '46', '33', '34', '38', '31', '63', '61', '92', '22', '80']


In [ ]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target_test = target_test.apply(lambda x: classes.index(x))

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
for number_of_attributes in range(MIN_number_of_attributes ,MAX_number_of_attributes+1):
  # Select a sub-dataframe from the main dataframe with a lower number of attributes
  df = df_raw[attributes_array[0:number_of_attributes]]
  df_test = df_test_raw[attributes_array[0:number_of_attributes]]
  #print(df_test)

  # Combine the data with possible classes
  numeric_feature_names = attributes_array[0:number_of_attributes]
  numeric_features = df[numeric_feature_names] # Data for training
  numeric_features_test = df_test[numeric_feature_names] # Data for testing

  # Converts pandas dataframe to tensorflow object
  numeric_features = tf.convert_to_tensor(numeric_features)
  #numeric_features_test = tf.convert_to_tensor(numeric_features_test) botei na célula abaixo

  # Normalize the data
  normalizer = tf.keras.layers.Normalization(axis=-1)
  normalizer.adapt(numeric_features)
  #normalizer.adapt(numeric_features_test)

  # Execute Trainning and Testing
  #model = get_basic_model()
  print('Starting training for',number_of_attributes,'attributes')
  train_accuracy = []
  test_acc = []
  for i in range(number_of_NN_repetitions):
    model = get_basic_model()
    history = model.fit(numeric_features, target, epochs=10, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
    one_train_accuracy = history.history['accuracy']

    test_loss, one_test_acc =  model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE)
    train_accuracy.append(one_train_accuracy[-1])
    test_acc.append(one_test_acc)
    print(f"{i+1}: train_accuracy:{train_accuracy} test_acc:{test_acc}")

  train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
  test_acc_mean = sum(test_acc) / len(test_acc)
  print(f'train_accuracy_mean:{train_accuracy_mean} test_acc_mean: {test_acc_mean}')
  # Date for report file
  current_date_and_time = datetime.date.today()
  current_date_and_time_string = str(current_date_and_time)
  # Report file content
  report = {
      "datetime": current_date_and_time_string,
      "training_dataset_shape":  df.shape,
      "test_dataset_shape": df_test.shape,
      "attributes": attributes_array[0:df.shape[1]],
      "number_of_attributes": number_of_attributes,
      "accuracy_train": train_accuracy_mean,
      "accuracy_test": test_acc_mean
  }
  # Writes into the file
  title = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/%s"%current_date_and_time_string + " With %d Attributes.json"%number_of_attributes
  with open(title, "w") as f:
    json.dump(report, f)
  with open(title, "r") as f:
    rep = json.load(f)
  #display(rep)

  #print(rep["datetime"])



Starting training for 34 attributes
2381/2381 - 3s - loss: 0.1697 - accuracy: 0.9594 - 3s/epoch - 1ms/step
1: train_accuracy:[0.9757068753242493] test_acc:[0.9593714475631714]
train_accuracy_mean:0.9757068753242493 test_acc_mean: 0.9593714475631714
Starting training for 35 attributes
2381/2381 - 3s - loss: 0.2057 - accuracy: 0.9551 - 3s/epoch - 1ms/step
1: train_accuracy:[0.979085385799408] test_acc:[0.9551279544830322]
train_accuracy_mean:0.979085385799408 test_acc_mean: 0.9551279544830322
